# Importing and Transforming raw DATA

In [1]:
import numpy as np
import pandas as pd
import os
from nltk.tokenize import RegexpTokenizer
from nltk.stem import *

folder = os.getcwd() ; print folder

/home/arda/Documents/youtube


In [2]:
pd_train = pd.read_csv(folder + '/data/train_sample.csv', sep=',', dialect=None, compression='infer', doublequote=True, 
            escapechar='\\', quotechar='"', quoting=0, skipinitialspace=False,
            lineterminator=None, header='infer', index_col=None, names=None,
            prefix=None, skiprows=None, skipfooter=None, skip_footer=0, na_values=None,
            na_fvalues=None, true_values=None, false_values=None, delimiter=None, converters=None,
            dtype=None, usecols=None, engine=None, delim_whitespace=False, as_recarray=False,
            na_filter=True, compact_ints=False, use_unsigned=False, low_memory=False, buffer_lines=None,
            warn_bad_lines=True, error_bad_lines=True, keep_default_na=True, thousands=None, comment=None,
            decimal='.', parse_dates=False, keep_date_col=False, dayfirst=False, date_parser=None,
            memory_map=False, float_precision=None, nrows=None, iterator=False, chunksize=None,
            verbose=False, encoding=None, squeeze=False, mangle_dupe_cols=True, tupleize_cols=False,
            infer_datetime_format=False, skip_blank_lines=True)


pd_test = pd.read_csv(folder + '/data/test_sample.csv', sep=',', dialect=None, compression='infer', doublequote=True, 
            escapechar='\\', quotechar='"', quoting=0, skipinitialspace=False,
            lineterminator=None, header='infer', index_col=None, names=None,
            prefix=None, skiprows=None, skipfooter=None, skip_footer=0, na_values=None,
            na_fvalues=None, true_values=None, false_values=None, delimiter=None, converters=None,
            dtype=None, usecols=None, engine=None, delim_whitespace=False, as_recarray=False,
            na_filter=True, compact_ints=False, use_unsigned=False, low_memory=False, buffer_lines=None,
            warn_bad_lines=True, error_bad_lines=True, keep_default_na=True, thousands=None, comment=None,
            decimal='.', parse_dates=False, keep_date_col=False, dayfirst=False, date_parser=None,
            memory_map=False, float_precision=None, nrows=None, iterator=False, chunksize=None,
            verbose=False, encoding=None, squeeze=False, mangle_dupe_cols=True, tupleize_cols=False,
            infer_datetime_format=False, skip_blank_lines=True)

In [3]:
print pd_train.columns

Index([u'video_category_id', u'title', u'description', u'published_at',
       u'viewCount', u'likeCount', u'dislikeCount', u'favoriteCount',
       u'commentCount', u'duration', u'dimension', u'definition', u'caption',
       u'licensedContent', u'topicIds', u'relevantTopicIds'],
      dtype='object')


In [4]:
import datetime
from dateutil import parser

def parse_TocpicId(dataframe):
    #create list of topicIds from raw string
    dataframe[u'topicIds'] = dataframe[u'topicIds'].apply(lambda r: r.split(";") if type(r)==str else [])
    return dataframe

def parse_relevantTopicIds(dataframe):
    #create list of relevantTopicIds from raw strings
    dataframe[u'relevantTopicIds'] = dataframe[u'relevantTopicIds'].apply(lambda r: r.split(";") if type(r)==str else [])
    return dataframe

def parse_published_at(dataframe):
    #parse the sate into datetime format 
    dataframe['published_at'] =  dataframe['published_at'].apply(lambda r: parser.parse(r[:19]) if len(r)==24 else datetime.datetime.now())
    return dataframe

In [5]:
pd_train = parse_TocpicId(pd_train)
pd_test = parse_TocpicId(pd_test)

pd_train = parse_relevantTopicIds(pd_train)
pd_test = parse_relevantTopicIds(pd_test)

In [6]:
pd_train = parse_published_at(pd_train)
pd_test = parse_published_at(pd_test)

In [10]:
pd_train

,video_category_id,title,description,published_at,viewCount,likeCount,dislikeCount,favoriteCount,commentCount,duration,dimension,definition,caption,licensedContent,topicIds,relevantTopicIds
0,19,Disneyland Railroad at night and first look at...,Disneyland Railroad at night and first look at...,2014-11-15 17:00:06,3826,154,0,0,26,PT8M2S,2d,hd,False,True,"[/m/02fzs, /m/0550hw, /m/028k8w]","[/m/010jjr, /m/01zx_2, /m/09b3v, /m/01jddz]"
1,19,David Kills Goliath,NaN,2013-09-27 10:56:23,30961,69,3,0,7,PT2M11S,2d,sd,False,False,"[/m/0cm45, /m/02cvn]","[/m/015j7, /m/06bvp]"
2,19,Downtown Disney | Walt Disney World | Parques ...,Qué quieres hacer cuando no estás en uno de lo...,2014-05-27 21:57:18,64558,68,12,0,7,PT2M24S,2d,hd,False,True,"[/m/081nh, /m/09b1k]","[/m/09b3v, /m/03tmlz, /m/0166ls, /m/04lm2k]"
3,19,The Renaissance Faire!,"This month, my mission for the Fiesta Movement...",2013-07-30 23:00:18,13776,662,6,0,193,PT4M33S,2d,hd,False,True,[/m/04dlk_],"[/m/04rlf, /m/06cvx]"
4,19,On board Quantum of the Seas - North Star | Ri...,Ich habe vom 31.10. - 02.11.2014 eine Kreuzfah...,2014-11-09 19:49:21,9233,57,0,0,11,PT5M6S,2d,hd,False,True,"[/m/0gh8bm0, /m/01tq2l, /m/038qps, /m/0463sh6]","[/m/06q74, /m/07nrkh, /m/04lm2k]"
5,19,SHARMEEN PITHI DANCES 1,NaN,2014-03-20 08:02:45,209208,413,36,0,12,PT8M18S,2d,hd,False,False,[/m/026bk],"[/m/01chg, /m/01gq53, /m/081hv, /m/02mtw0, /m/..."
6,19,The CAR Carrier : Indian Railways Freight Wagon,Catch the specially compiled video of the 3 Ge...,2014-10-09 17:48:12,20586,109,4,0,42,PT5M,2d,hd,False,True,"[/m/0303gp, /m/06d_3, /m/0k4j, /m/07jdr]","[/m/07bsy, /m/03x9c7, /m/03rk0, /m/07yv9, /m/0..."
7,19,"Florence, Italy: Michelangelo's David",One of Europe's great thrills is seeing Michel...,2013-07-03 15:00:18,53171,180,4,0,22,PT2M36S,2d,hd,True,True,"[/m/058w5, /m/0953j, /m/067s59]","[/m/031y2, /m/03rjj, /m/04lm2k]"
8,19,Dubrovnik and Balkan Side-Trips,Nestled inside its mighty ramparts is Dubrovni...,2013-08-19 23:15:19,40431,146,5,0,11,PT24M57S,2d,hd,True,True,"[/m/067s59, /m/0fbwq]","[/m/014dsx, /m/04lm2k, /m/07bxq, /m/02p4th, /m..."
9,19,McShrimp and Egg Burger,McDonald's in other countries have the weirdes...,2014-06-24 18:10:03,485701,6808,357,0,715,PT3M19S,2d,hd,False,False,"[/m/09syvmf, /m/0cdn1]","[/m/07gyp7, /m/033cnk, /m/02wbm, /m/02vt7ql]"
